In [1]:
import ipywidgets as widgets
import matplotlib as mpl
import pandas as pd
import random
import itertools
from matplotlib import colors
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings('ignore')
import pyensae
from pyensae.notebookhelper import folium_html_map
import folium

from IPython.display import HTML, display

plt.style.use('dark_background')
import plotly.graph_objects as go

from os.path import expanduser
import pandas as pd
import numpy as np
from tabulate import tabulate
import sys

import csv
from math import pi
import datetime
from itertools import groupby

plt.ioff()
%config InlineBackend.figure_format = 'svg'

DATA_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-'
CITIES_URL = 'https://raw.githubusercontent.com/klocey/COVID-dash/master/worldcities.csv'

In [2]:
class App_Scatter:
    
    def __init__(self, df):
        
        # x_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # y_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # z_indicator: Country/Region'
        # v_indicator: 'Province/State'
        
        self._df = df
        available_indicators1 = ['population size', 'Confirmed', 'Deaths', 'Recovered']
        self._x_dropdown = self._create_indicator_dropdown(available_indicators1, 1)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators1, 3)
        
        available_indicators2 = list(set(self._df['Country/Region']))
        available_indicators2.append('World')
        available_indicators2.sort(reverse=True)
        
        self._z_dropdown = self._create_indicator_dropdown(available_indicators2, 0)
        #available_indicators3 = ['Province/State']
        #self._v_dropdown = self._create_indicator_dropdown(available_indicators3, 0)
        
        self._plot_container = widgets.Output()
        #self._x_slider, x_slider_box = self._create_x_slider(
        #    0, 100000
        #)
        _app_container = widgets.VBox([
            widgets.VBox([self._x_dropdown, self._y_dropdown, self._z_dropdown]),
            self._plot_container,
            #x_slider_box
            ], 
            layout=widgets.Layout(align_items='center', flex='0 0 auto'))
        # 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset'
        self.container = widgets.VBox([
            widgets.HBox([
                _app_container,  
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
        
    @classmethod
    def from_url(cls, url):
        
        df_confirmed = pd.read_csv(DATA_URL + 'Confirmed.csv', index_col=False)
        df_confirmed['Province/State'].fillna('N/A',inplace=True)
        df_confirmed['location name'] = df_confirmed['Province/State'] + ', ' + df_confirmed['Country/Region']
        df_confirmed['type'] = 'Confirmed'

        df_deaths = pd.read_csv(DATA_URL + 'Deaths.csv', index_col=False)
        df_deaths['Province/State'].fillna('N/A',inplace=True)
        df_deaths['location name'] = df_deaths['Province/State'] + ', ' + df_deaths['Country/Region']
        df_deaths['type'] = 'Deaths'
        
        df_recovered = pd.read_csv(DATA_URL + 'Recovered.csv', index_col=False)
        df_recovered['Province/State'].fillna('N/A',inplace=True)
        df_recovered['location name'] = df_recovered['Province/State'] + ', ' + df_recovered['Country/Region']
        df_recovered['type'] = 'Recovered'
        
        
        ProvState = df_confirmed['Province/State']
        CountryRegion = df_confirmed['Country/Region']

        WorldCities = pd.read_csv(CITIES_URL, index_col=False)
        places = ProvState.tolist()

        popsizes = []
        for place in places:
            popsize = 0
            p_ls = ['Taiwan', 'San Diego County', 'Humboldt County', 'Sacramento County']
            s_ls = [23545963, 3095313, 132646, 1418788]
        
            if place in p_ls:
                i = p_ls.index(place)
                popsize = s_ls[i]
        
            else: 
                pattern = re.compile(r'\w\,')
                if pattern.findall(place):
                    place = ','.join(place.split(',')[:-1])    
    
                subdf = WorldCities[WorldCities.isin([place]).any(1)]
                popsize = sum(subdf['population'])
    
            popsizes.append(popsize)
    
        df_confirmed['population size'] = list(popsizes)
        df_deaths['population size'] = list(popsizes)
        df_recovered['population size'] = list(popsizes)
        
        df = pd.concat([df_confirmed, df_deaths, df_recovered])
        return cls(df)
        
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    '''
    def _create_x_slider(self, min_x, max_x):
        x_slider_label = widgets.Label('x-range: ')
        x_slider = widgets.IntRangeSlider(
            min=min_x, max=max_x,
            layout=widgets.Layout(width='300px')
        )
        x_slider.observe(self._on_change, names=['value'])
        x_slider_box = widgets.HBox([x_slider_label, x_slider])
        return x_slider, x_slider_box
    '''
    
    
    
    
    def _create_plot(self, x_indicator, y_indicator, z_indicator):
        
        # x_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # y_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # z_indicator: Country/Region'
        # v_indicator: 'Province/State'
        
        x = []
        y = []
        
        df_sub = self._df.dropna()
        if z_indicator == 'World':
            df_sub = self._df
            #print(df_sub.shape)
            #print(list(df_sub))
            #sys.exit()
        else:
            df_sub = self._df[self._df['Country/Region'] == z_indicator]
        
        if x_indicator == 'population size' and y_indicator == 'population size':
            df_sub = df_sub[df_sub['population size'] > 0]
            
            x = df_sub[df_sub['type'] == 'Confirmed']['population size']
            y = df_sub[df_sub['type'] == 'Confirmed']['population size']
            
        elif x_indicator == 'population size':
            df_sub = df_sub[df_sub['population size'] > 0]
            
            y = df_sub[df_sub['type'] == y_indicator].iloc[:,-4]
            x = df_sub[df_sub['type'] == y_indicator]['population size']
            
        elif y_indicator == 'population size':
            df_sub = df_sub[df_sub['population size'] > 0]
            
            x = df_sub[df_sub['type'] == x_indicator].iloc[:,-4]
            y = df_sub[df_sub['type'] == x_indicator]['population size']
            
        elif x_indicator != 'population size' and y_indicator != 'population size':
            x = df_sub[df_sub['type'] == x_indicator].iloc[:,-4]
            y = df_sub[df_sub['type'] == y_indicator].iloc[:,-4]
        
        plt.figure(figsize=(5, 4))
        plt.xlabel(x_indicator, size=12)
        plt.ylabel(y_indicator, size=12)
        plt.gca().tick_params(axis='both', which='major', labelsize=16)
        plt.scatter(x, y, s=80, c='r', alpha=0.3, linewidths=1, edgecolors='w')
        plt.tick_params(axis='both', labelsize=8)
        
        plt.xscale('symlog')
        plt.yscale('symlog')
        
        plt.xlim(0.1*min(x), 2*max(x))
        plt.ylim(0.1*min(y), 2*max(y))
        
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        z_indicator = self._z_dropdown.value
        #v_indicator = self._v_dropdown.value
        #x_range = self._x_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(x_indicator, y_indicator, z_indicator)
            plt.show()
    

In [3]:
class App_Time:
    
    def __init__(self, df):
        
        # x_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # y_indicator: 'population size'; 'Confirmed'; 'Deaths'; 'Recovered'
        # z_indicator: Country/Region'
        # v_indicator: 'Province/State'
        
        self._df = df
        available_indicators = list(set(self._df['Country/Region']))
        available_indicators.append('World')
        available_indicators.sort(reverse=True)
        
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._plot_container = widgets.Output()
        _app_container = widgets.VBox([
            widgets.VBox([self._x_dropdown]),
            self._plot_container,
            ], 
            layout=widgets.Layout(align_items='center', flex='0 0 auto'))
        self.container = widgets.VBox([
            widgets.HBox([
                _app_container,  
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
        
    @classmethod
    def from_url(cls, url):
        
        df_confirmed = pd.read_csv(DATA_URL + 'Confirmed.csv', index_col=False)
        df_confirmed['Province/State'].fillna('N/A',inplace=True)
        df_confirmed['location name'] = df_confirmed['Province/State'] + ', ' + df_confirmed['Country/Region']
        df_confirmed['type'] = 'Confirmed'

        df_deaths = pd.read_csv(DATA_URL + 'Deaths.csv', index_col=False)
        df_deaths['Province/State'].fillna('N/A',inplace=True)
        df_deaths['location name'] = df_deaths['Province/State'] + ', ' + df_deaths['Country/Region']
        df_deaths['type'] = 'Deaths'
        
        df_recovered = pd.read_csv(DATA_URL + 'Recovered.csv', index_col=False)
        df_recovered['Province/State'].fillna('N/A',inplace=True)
        df_recovered['location name'] = df_recovered['Province/State'] + ', ' + df_recovered['Country/Region']
        df_recovered['type'] = 'Recovered'
        
        
        ProvState = df_confirmed['Province/State']
        CountryRegion = df_confirmed['Country/Region']

        WorldCities = pd.read_csv(CITIES_URL, index_col=False)
        places = ProvState.tolist()

        popsizes = []
        for place in places:
            popsize = 0
            p_ls = ['Taiwan', 'San Diego County', 'Humboldt County', 'Sacramento County']
            s_ls = [23545963, 3095313, 132646, 1418788]
        
            if place in p_ls:
                i = p_ls.index(place)
                popsize = s_ls[i]
        
            else: 
                pattern = re.compile(r'\w\,')
                if pattern.findall(place):
                    place = ','.join(place.split(',')[:-1])    
    
                subdf = WorldCities[WorldCities.isin([place]).any(1)]
                popsize = sum(subdf['population'])
    
            popsizes.append(popsize)
    
        df_confirmed['population size'] = list(popsizes)
        df_deaths['population size'] = list(popsizes)
        df_recovered['population size'] = list(popsizes)
        
        df = pd.concat([df_confirmed, df_deaths, df_recovered])
        return cls(df)
        
        
    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    
    def _create_plot(self, x_indicator):
        fig = plt.figure(figsize=(5, 5))
        
        df_confirmed = self._df[self._df['type'] == 'Confirmed']
        if x_indicator != 'World':
            df_confirmed = df_confirmed[df_confirmed['Country/Region'] == x_indicator]
        df_timeseries = df_confirmed.iloc[:,4:-4]
        df_ts_sum = df_timeseries.sum().tolist()
        dates = list(df_timeseries)
        plt.fill_between(x=dates, y1=df_ts_sum, color='0.8', alpha=0.5, label = 'Confirmed')

        df_recovered = self._df[self._df['type'] == 'Recovered']
        if x_indicator != 'World':
            df_recovered = df_recovered[df_recovered['Country/Region'] == x_indicator]
        df_timeseries = df_recovered.iloc[:,4:-4]
        df_ts_sum = df_timeseries.sum().tolist()
        plt.fill_between(x=dates, y1=df_ts_sum, color='b', alpha=0.5, label = 'Recovered')

        df_deaths = self._df[self._df['type'] == 'Deaths']
        if x_indicator != 'World':
            df_deaths = df_deaths[df_deaths['Country/Region'] == x_indicator]
        df_timeseries = df_deaths.iloc[:,4:-4]
        df_ts_sum = df_timeseries.sum().tolist()
        plt.fill_between(x=dates, y1=df_ts_sum, color='r', alpha=0.9, label = 'Deaths')

        #plt.tick_params(axis='x', labelsize=8)
        #for label in ax.xaxis.get_ticklabels()[::2]:
        #    label.set_visible(False)

        ax = plt.gca()
        temp = ax.xaxis.get_ticklabels()
        temp = list(set(temp) - set(temp[::6]))
        for label in temp:
            label.set_visible(False)

        plt.legend(loc='upper left', frameon=False)
        
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        #x_range = self._x_slider.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(x_indicator)
            plt.show()
    

In [4]:
%config InlineBackend.figure_format = 'svg'

app1 = App_Scatter.from_url(DATA_URL)
app2 = App_Time.from_url(DATA_URL)
app3 = App_Scatter.from_url(DATA_URL)

# Put the views into a layout grid
grid = widgets.GridspecLayout(1, 3, height='480px', layout=widgets.Layout(justify_content='center'))
grid[0, 0] = app1.container
grid[0, 1] = app2.container
grid[0, 2] = app3.container

app_contents = [grid]

app = widgets.VBox(app_contents)
    
display(app)


In [5]:
def create_map():
    f = folium.Figure(width=1400, height=500)
    
    df_confirmed2 = pd.read_csv(DATA_URL + 'Confirmed.csv', index_col=False)
    df_confirmed2['Province/State'].fillna('N/A',inplace=True)
    #df_confirmed['location name'] = df_confirmed['Province/State'] + ', ' + df_confirmed['Country/Region']
    #df_confirmed['type'] = 'Confirmed'
        
    lats = df_confirmed2['Lat']
    lons = df_confirmed2['Long']
    
    labels = list(df_confirmed2)
    cases = df_confirmed2[labels[-1]].tolist()
    #print(cases)

    folium_map = folium.Map(location=[0, 0],
            zoom_start=2, tiles="CartoDB dark_matter").add_to(f)

    marker = folium.CircleMarker(location=[41.8781, -87.6298])

    marker.add_to(folium_map)

    for i in range(len(lons)):
        op = 0.5
        folium.Circle(location=[lats[i], lons[i]], radius=(cases[i]**0.5)*1000,
          color='crimson', fill=True, fill_color='crimson',
          opacity=op).add_to(folium_map)
            
    return folium_map
    
folium_map = create_map()
#folium_html_map(folium_map, width="100%")
folium_map